# API 모듈화

In [256]:
import requests
import pandas as pd

# 데이터 개수 측정
def DataCount(Decoding_key, pagenum, numofrows):
    url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'
    params ={'serviceKey' : Decoding_key, 'pageNo' : pagenum, 'numOfRows' : numofrows, 'type' : 'json' }
    response = requests.get(url, params=params)
    jsondata = response.json()
    
    total_count = jsondata['EarthquakeOutdoorsShelter'][0]['head'][0]['totalCount']
    total_count = int(total_count)
    return total_count
    
# 옥외 대피소 API 호출
def API_EarthQuake_json(Decoding_key, pagenum, numofrows):
    # API 호출
    url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'
    params ={'serviceKey' : Decoding_key, 'pageNo' : pagenum, 'numOfRows' : numofrows, 'type' : 'json' }
    response = requests.get(url, params=params)
    jsondata = response.json()
    # 호출 output 형태
    # 'EarthquakeOutdoorsShelter' : 결과 형태
    # 'row'
    
    # json 
    rows = jsondata['EarthquakeOutdoorsShelter'][1]['row']
    df = pd.DataFrame(rows)

    # Data_PreProcessing
    data = df[['vt_acmdfclty_nm','dtl_adres','fclty_ar','xcord','ycord']]

    # data rename
    data = data.rename(columns = {'vt_acmdfclty_nm' : '시설명', 'dtl_adres' : '상세주소', 'fclty_ar' : '시설면적', 'xcord' : '경도', 'ycord' : '위도'})

    # 상세주소 -> 광역시도, 시군구, 읍면동, 주소번호 split -> 상세주소 drop
    data[['광역시도','시군구', '읍면동', '주소번호']] = data['상세주소'].str.split(' ', expand=True)[[0,1,2,5]]
    data = data.drop(['상세주소'], axis=1)
    data = data[['광역시도','시군구', '읍면동', '주소번호', '경도', '위도', '시설명', '시설면적']]
    
    return data

# 전체 옥외 대피소 API 호출
def API_EarthQuake_Total(decoding, pagenum, numofrows):
    total_count = DataCount(decoding, pagenum, numofrows)

    if total_count % numofrows == 0:
        num = total_count//numofrows
    else:
        num = total_count//numofrows + 1
    
    total_df = pd.DataFrame({})
    
    for i in range(1, num+1):
        if i == num: # 마지막 페이지인 경우
            last_numofrows = total_count - (numofrows*(num-1))
            data = API_EarthQuake_json(decoding, i, last_numofrows)
        else: # 마지막 페이지 아닌경우
            data = API_EarthQuake_json(decoding, i, numofrows)
            
        total_df = pd.concat([data, total_df])

    total_df = total_df.sort_values(by = ['광역시도' , '시군구', '읍면동'])
    
    return total_df
# 전국 인원 API 호출
#def API_personnel(Decoding_key):

# 파라미터 변경사항

In [258]:
# API 서비스 키
decoding = 'tFlRbdLrGdbT07si0ZGVgNB2t7e1lRlQPzeq5uCeXyfkCZJ+8+CRvlRy1JLFEZXUdVqNxgmO1OaOcSDStjOhKQ=='
# pagenum
pagenum = 1
# 불러올 개수 (최대 1001개)
numofrows = 1000

total_df = API_EarthQuake_Total(decoding, pagenum, numofrows)

In [259]:
total_df

,광역시도,시군구,읍면동,주소번호,경도,위도,시설명,시설면적
69,강원특별자치도,강릉시,강동면,None,127.480805980668,37.2944046773841,강동초등학교,13604
797,강원특별자치도,강릉시,교동,1797-0,128.87967659489505,37.76979758522261,경포초등학교,11000
798,강원특별자치도,강릉시,교동,310-1,128.893021192654,37.7670051495753,교동초등학교,16721
68,강원특별자치도,강릉시,구정면,None,128.891387595211,37.7175835666904,구정초등학교,12065
803,강원특별자치도,강릉시,내곡동,620-0,128.87800647438,37.7400764324272,13호 어린이공원,2000
...,...,...,...,...,...,...,...,...
338,충청북도,충주시,용두동,411-1,127.878912,36.962977,달천초등학교 운동장,6900
412,충청북도,충주시,용산동,2109-0,127.937037,36.960190,충주남산초등학교 운동장,8450
413,충청북도,충주시,용산동,1671,127.944096,36.962318,용산초등학교 운동장,2500
414,충청북도,충주시,호암동,1075-0,127.92937110707157,36.954430137866844,남한강초등학교 운동장,2093


In [260]:
total_df.to_csv("total_earthquake.csv", index = False)

In [116]:
# data frame 생성 후 계속 append -> duplicated로 중복 제거 하는 코드


# SQL 연결

# API 호출 형태 확인

In [118]:
url = 'http://apis.data.go.kr/1741000/EmergencyAssemblyArea_Earthquake2/getArea1List'
params ={'serviceKey' : decoding, 'pageNo' : pagenum, 'numOfRows' : numofrows, 'type' : 'json' }
response = requests.get(url, params=params)
jsondata = response.json()

In [261]:
# jsondata['EarthquakeOutdoorsShelter'][0]

In [262]:
# jsondata['EarthquakeOutdoorsShelter'][1]
# output : {'row : [{옥외대피소 1 정보}, {옥외 대피소 2 정보}, ....]'}

In [122]:
# jsondata['EarthquakeOutdoorsShelter'][1]['row']
# ouput : [{옥외대피소 1 정보}, {옥외 대피소 2 정보}, ....]

In [263]:
# total_count = jsonData['EarthquakeOutdoorsShelter'][0]['head'][0]['totalCount']